# Installing The Environment

In order to do any kind of analysis we need the right tools.
Here we are quickly going to install DeepChem and jupyter to work in for the rest of the workshop


```bash
# Install DeepChem
git clone https://github.com/lilleswing/deepchem-slim.git
bash scripts/install_deepchem_conda.sh future_of_care
source activate future_of_care
python setup.py install
cd ..

# Install jupyter
conda install jupyter jupyterlab

#Start your Notebook
jupyter lab
```

## What We Are Doing Today

https://pubs.acs.org/doi/abs/10.1021/acs.molpharmaceut.8b00110
This paper came across my radar last week.  
TODO (Some filler about why p450 is important)





# Grabbing The Data

Training Set
https://pubchem.ncbi.nlm.nih.gov/bioassay/1851

Test Sets
* https://pubchem.ncbi.nlm.nih.gov/bioassay/410
* https://pubchem.ncbi.nlm.nih.gov/bioassay/883
* https://pubchem.ncbi.nlm.nih.gov/bioassay/899
* https://pubchem.ncbi.nlm.nih.gov/bioassay/891
* https://pubchem.ncbi.nlm.nih.gov/bioassay/884


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/AID_1851_datatable_all.csv')
curve_columns = list(filter(lambda x: x.find('CurveClass') > 0, df.columns.tolist()))
score_columns = list(filter(lambda x: x.find('Score') > 0, df.columns.tolist()))
ID_COLUMN = ['PUBCHEM_CID']
df = df[ID_COLUMN+curve_columns + score_columns]

/home/leswing/miniconda3/envs/cornell/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,7,9,10,12,13,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,42,43,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,66,67,69,70,71,72,73,74,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,93,94,96,97,98,99,100,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,120,121,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns.tolist()

['PUBCHEM_CID',
 'p450-cyp2c19-Fit_CurveClass',
 'p450-cyp2d6-Fit_CurveClass',
 'p450-cyp3a4-Fit_CurveClass',
 'p450-cyp1a2-Fit_CurveClass',
 'p450-cyp2c9-Fit_CurveClass',
 'Activity Score',
 'Activity Score.1',
 'Activity Score.2',
 'Activity Score.3',
 'Activity Score.4']

In [4]:
import math
def label_column(score, curve_class):
    # Convert Units
    score = float(score)
    curve_class = str(abs(float(curve_class)))
    
    if score >= 40 and curve_class in ("1.1", "1.2", "2.1"):
        return 1
    if curve_class in ("4.0") and score==0:
        return 0
    return -1

def read_smiles_lookup_map(fname):
    with open(fname) as fin:
        lines = fin.readlines()
        lines = [x.strip().split(',') for x in lines]
        d = {x[0]:x[1] for x in lines}
        return d
        

def df_to_single_task(df, score_column, curve_column):
    cids = df['PUBCHEM_CID'].tolist()[3:]
    curve_classes = df[curve_column].tolist()[3:]
    scores = df[score_column].tolist()[3:]
    
    smiles_lookup = read_smiles_lookup_map('data/smiles_lookup.csv')
    table = []
    for cid, curve_class, score in zip(cids, curve_classes, scores):
        if pd.isna(cid) or pd.isna(curve_class) or pd.isna(score):
            continue
        cid = str(int(cid))
        if cid not in smiles_lookup:
            continue
        smiles = smiles_lookup[cid]
        table.append([smiles,curve_class,score])
        
    final_table = []
    for smile, curve_class, score in table:
        label = label_column(score, curve_class)
        if label == -1:
            continue
        final_table.append([smile, str(label)])
    return final_table


In [5]:
table = df_to_single_task(df, 'Activity Score', 'p450-cyp2c19-Fit_CurveClass')
with open('assets/2c19_train.csv', 'w') as fout:
    for line in table:
        line = ",".join(line)
        fout.write("%s\n" % line)

In [6]:
# This doesn't line up with the paper!
# This is a very common thing in this field.....
y = [float(x[1]) for x in table]
len(y), sum(y),  len(y) - sum(y)

(10919, 5103.0, 5816.0)

In [7]:
import pandas as pd
df = pd.read_csv('data/AID_899_datatable_all.csv')
curve_columns = list(filter(lambda x: x.find('CurveClass') > 0, df.columns.tolist()))
score_columns = list(filter(lambda x: x.find('SCORE') > 0, df.columns.tolist()))
ID_COLUMN = ['PUBCHEM_CID']
df = df[ID_COLUMN+curve_columns + score_columns]

In [8]:
df.columns.tolist()

['PUBCHEM_CID', 'Fit_CurveClass', 'PUBCHEM_ACTIVITY_SCORE']

In [9]:
table = df_to_single_task(df, 'PUBCHEM_ACTIVITY_SCORE', 'Fit_CurveClass')

used_smiles = set()
with open('assets/2c19_train.csv') as fin:
    lines = [x.strip().split(',') for x in fin.readlines()]
    used = set([x[0] for x in lines])
    used_smiles.update(used)

unique_table = []
for line in table:
    if line[0] in used_smiles:
        continue
    unique_table.append(line)
table = unique_table
with open('assets/2c19_test.csv', 'w') as fout:
    for line in table:
        line = ",".join(line)
        fout.write("%s\n" % line)

In [10]:
y = [float(x[1]) for x in table]
len(y), sum(y),  len(y) - sum(y)

(722, 172.0, 550.0)